# 🎻 PyKwant Tutorial: Financial Instruments

This notebook introduces the `pykwant.instruments` module.

In object-oriented libraries, an instrument usually "knows" how to price itself (e.g., `bond.price()`). In **PyKwant**, we follow a functional approach:

1. **Instruments are Data**: They are immutable `dataclasses` containing only the contract definition.

2. **Pricing is a Function**: Logic is applied externally (e.g., `price_instrument(bond, curve, date)`).

## 1. Setup and Imports

In [ ]:
from datetime import date

from pykwant import dates, instruments, rates


# Helper to visualize cash flows nicely
def print_flows(flows):
    print(f"{'Date':<15} | {'Type':<10} | {'Amount':>10}")
    print("-" * 40)
    for cf in flows:
        print(f"{str(cf.payment_date):<15} | {cf.type:<10} | {cf.amount:>10.2f}")

## 2. Defining a Fixed Rate Bond

Let's model a **5-Year Corporate Bond**:

* **Face Value**: 100.0

* **Coupon**: 5% (Annual)

* **Maturity**: 5 Years

**Note**: We must strictly provide a `Calendar`. The instrument needs this data to define how payment dates roll if they fall on holidays.

In [13]:
# 1. Define the Calendar (e.g., Target/Milan)
# Holidays: New Year and Christmas
cal = dates.Calendar(holidays=frozenset([date(2025, 1, 1), date(2025, 12, 25)]), weekends=(6, 7))

# 2. Define the Instrument (Immutable Data)
bond = instruments.FixedRateBond(
    face_value=instruments.Money(100.0),
    coupon_rate=0.05,
    start_date=date(2025, 1, 1),
    maturity_date=date(2030, 1, 1),
    frequency_months=12,  # Annual coupons
    day_count=dates.thirty_360,  # Standard Corporate convention
    calendar=cal,
)

print("--- Instrument Definition ---")
print(bond)

--- Instrument Definition ---
FixedRateBond(face_value=100.0, coupon_rate=0.05, start_date=datetime.date(2025, 1, 1), maturity_date=datetime.date(2030, 1, 1), frequency_months=12, day_count=<function thirty_360 at 0x000002B3C870E610>, calendar=Calendar(holidays=frozenset({datetime.date(2025, 1, 1), datetime.date(2025, 12, 25)}), weekends=(6, 7)))


## 3. Cash Flow Generation

Before pricing, we can inspect the deterministic schedule of cash flows. The function `generate_cash_flows` transforms the Bond definition into a stream of payment objects.

In [14]:
flows = instruments.generate_cash_flows(bond)

print("\n--- Scheduled Cash Flows ---")
print_flows(flows)


--- Scheduled Cash Flows ---
Date            | Type       |     Amount
----------------------------------------
2026-01-01      | coupon     |       5.00
2027-01-01      | coupon     |       5.00
2028-01-03      | coupon     |       5.03
2029-01-01      | coupon     |       4.97
2030-01-01      | coupon     |       5.00
2030-01-01      | principal  |     100.00


## 4. Market Environment

To price the bond, we need a Yield Curve. Let's create a simple flat curve at **4%**.

In [15]:
valuation_date = date(2025, 4, 1)  # 3 months after issue


def flat_curve(d: date) -> float:
    t = dates.act_365(valuation_date, d)
    # Continuous compounding: DF = exp(-r * t)
    return rates.compound_factor(-0.04, t, frequency=0)


# 2. Valuation Date
valuation_date = date(2025, 1, 1)

# 3. Calculate NPV (Dirty Price)
dirty_price = instruments.price_instrument(bond, flat_curve, valuation_date)

print(f"\n--- Valuation at {valuation_date} ---")
print("Market Rate: 4% Flat")
print(f"Dirty Price: {dirty_price:.4f}")


--- Valuation at 2025-01-01 ---
Market Rate: 4% Flat
Dirty Price: 104.0717


## 5. Valuation: Dirty Price (NPV)

The `price_instrument` function calculates the **Net Present Value (NPV)** of future cash flows. In bond markets, this is often called the **Dirty Price** because it includes accrued interest.

In [16]:
dirty_price = instruments.price_instrument(
    instrument=bond, curve=flat_curve, valuation_date=valuation_date
)

print(f"Valuation Date: {valuation_date}")
print(f"Dirty Price:    {dirty_price:.4f}")

Valuation Date: 2025-01-01
Dirty Price:    104.0717


## 6. Accrued Interest & Clean Price

Traders usually quote the **Clean Price**, which strips out the interest accumulated since the last coupon.

$$\text{Clean Price} = \text{Dirty Price} - \text{Accrued Interest}$$

We are pricing on April 1st, exactly 3 months (0.25 years) into the coupon period. We expect roughly $0.25 \times 5\% = 1.25$ in accrued interest.

In [17]:
# 1. Calculate Accrued Interest
accrued = instruments.accrued_interest(bond, valuation_date)

# 2. Calculate Clean Price
clean_p = instruments.clean_price(bond, flat_curve, valuation_date)

print("\n--- Price Breakdown ---")
print(f"Dirty Price (NPV):   {dirty_price:>10.4f}")
print(f"(-) Accrued Interest:{accrued:>10.4f}")
print("-" * 32)
print(f"(=) Clean Price:     {clean_p:>10.4f}")

# Verification
assert abs((clean_p + accrued) - dirty_price) < 1e-9
print("\n✅ Accounting check passed.")


--- Price Breakdown ---
Dirty Price (NPV):     104.0717
(-) Accrued Interest:    0.0000
--------------------------------
(=) Clean Price:       104.0717

✅ Accounting check passed.


## 7. Extensibility: Equity Options

We recently added support for **European Options**. Even though the pricing logic (Black-Scholes) lives in the `equity` module, the instrument definition lives here in `instruments` to centralize data structures.

This demonstrates how the library handles polymorphism via Data Classes.

In [18]:
call_option = instruments.EuropeanOption(
    asset_name="PYK Tech",
    strike=instruments.Money(100.0),
    expiry_date=date(2026, 1, 1),
    call_put="call",
)

print("\n--- New Instrument Type ---")
print(call_option)


--- New Instrument Type ---
EuropeanOption(asset_name='PYK Tech', strike=100.0, expiry_date=datetime.date(2026, 1, 1), call_put='call')


If we try to price this using the generic `instruments.price_instrument`, the library safely raises an error, guiding us to use the specific equity module (which ensures we don't accidentally price an option like a bond).

In [19]:
try:
    instruments.price_instrument(call_option, flat_curve, valuation_date)
except NotImplementedError as e:
    print(f"\nExpected Error: {e}")


Expected Error: Option pricing is handled in the 'equity' module via 'black_scholes_price'.
